In [ ]:
import pandas as pd #表格處理
import matplotlib as plt #畫圖

#MA
def MA(s,l,df):
    df['ma_s'] = df.iloc[:,4].rolling(s).mean() #收盤價計算 S日均線 短均線
    df['ma_l'] = df.iloc[:,4].rolling(l).mean() #L長均線
    #新增 MA交易訊號
    df['ma_sign'] = 0
    #設定 交叉向上 標示 =1
    df['ma_sign'][(df['ma_s'].shift(1) < df['ma_l'].shift(1) ) & (df['ma_s'] > df['ma_l']) ] = 1
    #向下 標示 =-1
    df['ma_sign'][(df['ma_s'].shift(1) < df['ma_l'].shift(1) ) & (df['ma_s'] < df['ma_l']) ] = -1

#RSI
def RSI(d,df):
    #以今天的收盤價 - 前役天的收盤，算出一個序列，命為X
    x = df.iloc[:,4].diff() #計算變值
    #先將X中小魚0的值預設為0然後每d天對X取平均
    #再將x 中富庶改為正值,然後每d天對x取平均
    #將兩個平均數相除後*100，算出序列，及RSI，命['rsi']
    #where 函數，用來尋找符合條件值，把不符合地直指定為其他數值
    df['rsi'] = 100*x.where( x>0,0 ).rolling(10).mean() / x.where(x>0,-x).rolling(10).mean()
    #rsi 訊號
    df['rsi_sign'] = 0
    #RSI超賣訊號=1
    df['rsi_sign'][(df['rsi']<20)] = 1
    #超買 =-1
    df['rsi_sign'][(df['rsi']>80)] = -1
    
#定義出場函數，
def outp(df,r,b,price,i):
    #r資金存量，b多空方設定，多方=1，空方=-1
    #price=? 選擇開收盤
    r = r + b*df.iloc[i,price]*1000      #*1000契約成數
    df['ret'].iloc[i] = r #結算紀錄
    r=0
    b=0 
    return(r,b)

#進場函數
def inp(df,r,b,i):
    #r=成本,b=多空方設定，多方=1，空方=-1
    df['sign'].iloc[i] = b #進場時紀錄多空
    r = r-b*df.iloc[i,1]*1000 #設定多方買進與空方賣出
    return (r,b)

#定義當日結算與停損停利函數
def stop(df,wsp,lsp,r,b,i):
    #r=成本,b=多空設定,多方=1,空方=-1
    mm = r + b*df.iloc[i,4]*1000    #當日結算(收盤價)
    mp = mm / (-b*r)             #以 當日結算價/進場成本 計算
    #停損停利
    if mp>wsp or mp<lsp:
        (r,b)=outp(df,r,b,1,i+1)
    return(r,b)

#算績效
def 